In [2]:
import tensorflow as tf
import numpy as np

# Using session "InteractiveSession" instead
# def runTensor(tensor):
#     with tf.Session() as sess:
#         return sess.run(tensor)
sess = tf.InteractiveSession()

### Variables

- I'd recommend to always use ```tf.get_variable(...)``` -- it will make it way easier to refactor your code if you need to share variables at any time.



- In Notebook, need to use ```tf.reset_default_graph()``` to reset graph
- Can use ```xavier_initializer```

I can find two main differences between one and the other:

 1. First is that ```tf.Variable``` will always create a new variable, whether ```tf.get_variable``` gets from the graph an existing variable with those parameters, and if it does not exists, it creates a new one.

 2. ```tf.Variable``` requires that an initial value be specified.

[From Stackoverflow](https://stackoverflow.com/questions/37098546/difference-between-variable-and-get-variable-in-tensorflow)

In [18]:
tf.reset_default_graph()
W = tf.get_variable("w", shape=(3, 1), initializer=tf.contrib.layers.xavier_initializer())
init_g = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_g)
    print(W.eval())

[[ 1.0927178 ]
 [ 0.07955277]
 [-0.41374165]]


variable_scope

In [21]:
with tf.variable_scope("one"):
    a = tf.get_variable("v", [1]) #a.name == "one/v:0"
with tf.variable_scope("one", reuse = True):
    c = tf.get_variable("v", [1]) #c.name == "one/v:0"

In [ ]:
def get_scope_variable(scope, name, shape=None):
    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
        v = tf.get_variable(name, shape, initializer=tf.contrib.layers.xavier_initializer())
    return v

### Resize

In [2]:
a = tf.reshape([1,2,3,4],shape=[2,2])
print('a=', a.eval())

b = tf.reshape([1,2,3,4,5,6],shape=[-1,2]) # -1 means what ever that fit
print('b=', b.eval())

a= [[1 2]
 [3 4]]
b= [[1 2]
 [3 4]
 [5 6]]


### Convolution

In [3]:
x=np.array([[[[1,2,3],[4,5,6]],
           [[7,8,9],[10,11,12]]]],dtype=np.float32)
W=np.array([[[[1],[1],[1]]]],dtype=np.float32)

print('x.shape = ', x.shape)
print('W.shape = ', W.shape)

# conv
#  - stride = [batch, height(row), width(col), channel] >>> typically batch and channel should be 1 
#             because no skip batch and apply to all channel
#  - padding = 'SAME' add padding 0
conv = tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')
print('conv.shape = ', conv.shape)

x.shape =  (1, 2, 2, 3)
W.shape =  (1, 1, 3, 1)
conv.shape =  (1, 2, 2, 1)


#### Pooling

In [4]:
# [batch, height, width, channels]
x = list(range(1,25))
x = np.reshape(x,[1,2,3,4])

print('batch   :',len(x))
print('height  :',len(x[0]))
print('width   :',len(x[0][0]))
print('channels:',len(x[0][0][0]))
# # Try to adjust height/width/channel of ksize and strides to make you more understand.
# >>> typically batch and channel should be 1
max_pool = tf.nn.max_pool( x, ksize=[1, 2, 1, 1], # size of window : [batch, height, width, channels]
                            strides=[1, 1, 1, 1], # stride (skip) : [batch, height, width, channels]
                            padding='SAME')
max_pool.eval()

batch   : 1
height  : 2
width   : 3
channels: 4


array([[[[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]],

        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]]])

### Dropout
```python
tf.nn.dropout(full_layer_one, keep_prob=hold_prob)
```
It's only dropout in layer, not the whole network.

### Basic

In [23]:
ph = tf.placeholder(tf.float32, name="yo")
x = tf.Variable(1)
c = tf.constant(5.0)

### Util

In [6]:
tf.reduce_mean([[1,2,3],[4,5,6]],axis=1).eval()

array([2, 5], dtype=int32)

In [7]:
tf.argmax([[1,2,3],[7,5,6]],1).eval()

array([2, 0])

### Get Weight

In [9]:
X_ = tf.placeholder(tf.float64, [None, 5], name="Input")
Y_ = tf.placeholder(tf.float64, [None, 1], name="Output")

X = np.random.randint(1,10,[10,5])
Y = np.random.randint(0,2,[10,1])

with tf.variable_scope("LogReg"):
    # reuse = True : just for rerunning this snipplet
    pred = tf.contrib.layers.fully_connected(X_, 1, activation_fn=tf.nn.sigmoid, scope = 'fc1', reuse=tf.AUTO_REUSE)
    loss = tf.losses.mean_squared_error(labels=Y_, predictions=pred)
    training_ops = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

with tf.Session() as sess:

    all_vars= tf.global_variables()
    def get_var(name):
        for i in range(len(all_vars)):
            if all_vars[i].name.startswith(name):
                return all_vars[i]
        return None
    fc1_var = get_var('LogReg/fc1/weights')

    sess.run(tf.global_variables_initializer())    
    for i in range(200):
        _,fc1_var_np = sess.run([training_ops,fc1_var], feed_dict={
        X_: X,
        Y_: Y 
        })
        print(fc1_var_np)

[[ 0.64561473]
 [ 0.16171492]
 [-0.56667975]
 [-0.51069586]
 [ 0.16061438]]
[[ 0.64128838]
 [ 0.15659153]
 [-0.56902042]
 [-0.51227018]
 [ 0.15805164]]
[[ 0.63670926]
 [ 0.15131068]
 [-0.5715858 ]
 [-0.51399054]
 [ 0.15527839]]
[[ 0.63187181]
 [ 0.14587935]
 [-0.57437236]
 [-0.51586501]
 [ 0.15229611]]
[[ 0.62677544]
 [ 0.1403097 ]
 [-0.57737017]
 [-0.51790027]
 [ 0.14911173]]
[[ 0.62142593]
 [ 0.13461987]
 [-0.5805618 ]
 [-0.52010089]
 [ 0.14573867]]
[[ 0.61583676]
 [ 0.12883431]
 [-0.58392167]
 [-0.52246853]
 [ 0.14219744]]
[[ 0.61002998]
 [ 0.12298357]
 [-0.58741616]
 [-0.52500112]
 [ 0.13851567]]
[[ 0.6040366 ]
 [ 0.11710347]
 [-0.59100461]
 [-0.5276921 ]
 [ 0.13472741]]
[[ 0.597896  ]
 [ 0.1112334 ]
 [-0.59464127]
 [-0.53052994]
 [ 0.13087159]]
[[ 0.59165461]
 [ 0.10541426]
 [-0.59827816]
 [-0.53349791]
 [ 0.12698977]]
[[ 0.5853637 ]
 [ 0.09968603]
 [-0.60186829]
 [-0.5365744 ]
 [ 0.1231236 ]]
[[ 0.57907674]
 [ 0.09408567]
 [-0.60536864]
 [-0.53973363]
 [ 0.11931234]]
[[ 0.5728465

In [4]:
a = tf.Variable(tf.zeros([2,4]), name="W")
b = tf.Variable(tf.zeros([4]), name="b")
c = a+b

tf.global_variables_initializer().run()
sess.run(c)

# Get all weight
tvars = tf.trainable_variables()
tvars_vals = sess.run(tvars)
for var, val in zip(tvars, tvars_vals):
    print(var.name, val)

W:0 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]]
b:0 [0. 0. 0. 0.]
W_1:0 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]]
b_1:0 [0. 0. 0. 0.]


### a. get weight from manual code

In [ ]:
# ex. 1
w_b = [v for v in tf.global_variables() if v.name == 'layer_1/w:0' or v.name == 'layer_1/b:0']
w_val = w_b[0].eval()
b_val = w_b[1].eval()
w_val, b_val

In [ ]:
# ex. 2
model_vars = tf.trainable_variables()
d_A_vars = [var for var in model_vars if 'd_A' in var.name]

### b. get weight from model

In [ ]:
var_names = tf.estimator.LinearClassifier.get_variable_names(train)
for name in var_names:
    var_value = tf.estimator.LinearClassifier.get_variable_value(train, name)
    print(name, var_value)

### Stack

In [7]:
x = tf.constant([1, 4])
y = tf.constant([2, 5])
z = tf.constant([3, 6])
print(tf.stack([x, y, z]).eval()) # [[1, 4], [2, 5], [3, 6]] (Pack along first dim.)
print(tf.stack([x, y, z], axis=1).eval())  # [[1, 2, 3], [4, 5, 6]]

[[1 4]
 [2 5]
 [3 6]]
[[1 2 3]
 [4 5 6]]


### Cross Entropy
From this [link](https://stackoverflow.com/questions/46291253/tensorflow-sigmoid-and-cross-entropy-vs-sigmoid-cross-entropy-with-logits?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa)

```python
p = tf.placeholder(tf.float32, shape=[None, 5])
logit_q = tf.placeholder(tf.float32, shape=[None, 5])
q = tf.nn.sigmoid(logit_q)

feed_dict = {
  p: [[0, 0, 0, 1, 0],
      [1, 0, 0, 0, 0]],
  logit_q: [[0.2, 0.2, 0.2, 0.2, 0.2],
            [0.3, 0.3, 0.2, 0.1, 0.1]]
}

prob2 = p * -tf.log(q) + (1 - p) * -tf.log(1 - q)
prob3 = p * -tf.log(tf.sigmoid(logit_q)) + (1-p) * -tf.log(1-tf.sigmoid(logit_q))
prob4 = tf.nn.sigmoid_cross_entropy_with_logits(labels=p, logits=logit_q)

# prob2 - prob4 are the same result
print(prob2.eval(feed_dict))
print(prob3.eval(feed_dict))
print(prob4.eval(feed_dict))
```

## Tensorboard

In [5]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
show_graph(tf.get_default_graph())

### Helper function
#### 1. dense (fully connected)

In [ ]:
tf.layers.dense(x, 10, tf.nn.relu)

#### 2. relu

In [ ]:
tf.nn.relu(x)

#### 3 lose

In [ ]:
tf.losses.mean_squared_error(y_true, y_pred)